In [20]:
pip install faker

In [21]:
import csv
import random
import pandas as pd
import csv
from faker import Faker
from datetime import date, timedelta

In [22]:
# Users creation
user_data = pd.read_csv("/content/User_data.csv" ,names=['Name', 'phoneNumber', 'password'])

user_data['userid'] = [i for i in range(1, len(user_data)+1)]

user_data.to_csv("New_user_data.csv", header=False, index=False, columns=['userid', 'Name', 'phoneNumber', 'password'])

In [32]:
#1. Bankcard creation

def generate_card_number():
    return "{:12.0f}".format(float(random.randint(10**(15), (10**16)-1)))

def create_csv_with_user_and_card(input_csv_file, output_csv_file):
    with open(input_csv_file, 'r') as infile:
        reader = csv.reader(infile)
        next(reader)
        user_ids = [row[0] for row in reader]

    with open(output_csv_file, 'w', newline='') as outfile:
        writer = csv.writer(outfile)
        cardset = set()
        for user_id in user_ids:
            card_number = str(generate_card_number())
            if card_number not in cardset:
              writer.writerow([card_number, user_id])
              cardset.add(card_number)

input_csv_file = '/content/User_data.csv'
output_csv_file = 'Bankcard_details.csv'
create_csv_with_user_and_card(input_csv_file, output_csv_file)

In [24]:
data_1 = pd.read_csv(input_csv_file)

In [25]:
#2. User address creation

fake = Faker()
def generate_fake_data(num_rows, user_ids):
    data = []
    for i in range(1, num_rows + 1):
        user_id = random.choice(user_ids) if user_ids else None
        data.append([i,user_id,fake.street_address(),str(random.randint(100000, 999999)),])
    return data

total_users = len(data_1)
user_ids = []
with open('/content/User_data.csv', 'r') as csvfile:
    reader = csv.reader(csvfile)
    next(reader)
    user_ids = [row[0] for row in reader]

fake_data = generate_fake_data(total_users, user_ids)
filename = 'User_address.csv'
with open(filename, 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerows(fake_data)

In [26]:
#3. Product Database

def read_data_from_csv(filename):
    data = []
    with open(filename, 'r') as csvfile:
        reader = csv.DictReader(csvfile)
        for row in reader:
            data.append([row['Description'],row['UnitPrice']])
    return data

existing_data_filename = '/content/online_retail.csv'
product_data = read_data_from_csv(existing_data_filename)
pid_value = 1

output_filename = 'Product_data.csv'
with open(output_filename, 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    for row in product_data:
        writer.writerow([pid_value] + row)
        pid_value += 1

In [46]:
users_data = pd.read_csv("/content/User_data.csv", names=['userId', 'name', 'phoneNumber', 'password'])
products_data = pd.read_csv("/content/Product_data.csv", names=['pid', 'Name', 'price'])

#create orders
orders_data = pd.DataFrame(columns=['orderNumber', 'paymentState', 'totalAmount'])
orderitems_data = pd.DataFrame(columns=['itemid', 'pid', 'price'])
contain_data = pd.DataFrame(columns=['orderNumber', 'itemid', 'quantity'])
paymentStates = ['Paid', 'Pending', 'Declined']
contain_index = 1
orderitems_index = 1
for i in range(2800):
  user_index = random.randint(1, len(users_data))
  totalAmount = 0.0
  paymentState = ''
  quantity = int()
  # Items per order
  items_per_order = random.randint(1, 6)
  for j in range(items_per_order):
    product_index = random.randint(1, len(products_data)-1)
    price = float(products_data.loc[product_index, 'price'])
    quantity = random.randint(1, 5)
    totalAmount += float(price * quantity)
    paymentState = paymentStates[random.randint(0, 2)]

    orderitems_data.loc[orderitems_index] = [str(orderitems_index), product_index, price]
    contain_data.loc[contain_index] = [i, orderitems_index, quantity]
    orderitems_index += 1
    contain_index += 1

  orders_data.loc[i] = [i, paymentState, totalAmount]

orderitems_data.to_csv("orderitems.csv", header=False, index=False)
orders_data.to_csv("orders.csv", header=False, index=False)
contain_data.to_csv("contain.csv", header=False, index=False)





In [45]:
# create reviews
reviews_data = pd.DataFrame(columns=['user_id', 'pid', 'grade'])
for i in range(2500):
  user_index = random.randint(1, len(users_data)-1)
  product_index = random.randint(1, len(products_data)-1)
  grade = float(random.randint(1, 10)/2)

  reviews_data.loc[i] = [str(user_index), product_index, grade]

reviews_data.to_csv("Reviews.csv", header=False, index=False)


In [29]:
address_data = pd.read_csv('/content/User_address.csv', names=['addrid', 'userid', 'streetaddr', 'postCode'])
# create Deliver_To
deliver_to_data = pd.DataFrame(columns=['addrid', 'orderNumber', 'TimeDelivered'])

for i in range(len(orders_data)):
  address_index = random.randint(1, len(address_data)-1)
  random_year = random.randint(2022, 2023)
  random_month = random.randint(1, 12)
  random_day = random.randint(1, 28)
  delivered_date = date(random_year, random_month, random_day)

  deliver_to_data.loc[i] = [address_index, i, delivered_date]
deliver_to_data.to_csv('Deliver_to.csv', index=False, header=False)

In [36]:
#payment dataset
orders_data = pd.read_csv('/content/orders.csv', names=['orderNumber', 'paymentStatus', 'totalAmount'])
bankcard_data = pd.read_csv('/content/Bankcard_details.csv', names=['bankcardNumber', 'userid'])
bankcards = len(bankcard_data)

payment_data = pd.DataFrame(columns=['orderNumber', 'bankCardNumber'])

for i in range(len(orders_data)):
  card = bankcard_data.loc[random.randint(0, bankcards-1), 'bankcardNumber']
  payment_data.loc[i] = [orders_data.loc[i, 'orderNumber'], card]

payment_data.to_csv('Payment_data.csv', index=False, header=False)
